# Concept induction
This notebook offers sample [EDSL](https://docs.expectedparrot.com) code for using language models to identify concepts in unstructured texts, then generate criteria for the concepts, and then apply the criteria to evaluate the texts.

This idea is inspired by the recent paper: [Concept Induction: Analyzing Unstructured Text with High-Level Concepts Using LLooM](https://hci.stanford.edu/publications/2024/Lam_LLooM_CHI24.pdf)

## Technical setup
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Identify concepts
We start by creating a general question prompting the respondent (a language model) to identify concepts in a given text.

EDSL comes with a variety of [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model. `QuestionList` may be appropriate where we want the response to be formatted as a list of strings:

In [1]:
from edsl import QuestionList

q_concepts = QuestionList(
    question_name="concepts",
    question_text="Identify the key concepts in the following text: {{ text }}",
    # max_list_items = # Optional
)

We might also want to ask some other questions about our data at the same time (a data labeling task). For example:

In [2]:
from edsl import QuestionMultipleChoice

q_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="Identify the sentiment of this text: {{ text }}",
    question_options=["Negative", "Neutral", "Positive"],
)

We parameterize the questions in order to run them for each of our texts. This is done with `Scenario` objects that we create for our data (here, some recent tweets by Pres. Biden):

In [3]:
# Replace with your data
texts = [  # POTUS recent tweets
    "Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",
    "We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",
    "Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.",
    "Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.",
    "This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'",
    "The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.",
    "Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.",
    "Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.",
    "Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.",
    "With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with.",
]
len(texts)

10

In [4]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("text", texts)
scenarios

,text
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C."
1,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom."
2,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.
3,"Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States."
4,"This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'"
5,The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.
6,"Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day."
7,Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.
8,"Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients."
9,"With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with."


Next we combine the questions into a survey in order to administer them together (asynchronously by default, or according to any skip/stop rules or other logic that we want to add--learn more about `Survey` methods in our [documentation](https://docs.expectedparrot.com/en/latest/surveys.html)):

In [5]:
from edsl import Survey

survey = Survey(questions=[q_concepts, q_sentiment])

We add the scenarios to the survey and then run it to generate a dataset of results:

In [6]:
results = survey.by(scenarios).run()

Job UUID,92a02c28-48e1-4557-9108-fcd9d6387d9d
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/92a02c28-48e1-4557-9108-fcd9d6387d9d
Exceptions Report URL,None
Results UUID,1c820593-d578-4f12-a784-6c5b5dffa344
Results URL,https://www.expectedparrot.com/content/1c820593-d578-4f12-a784-6c5b5dffa344


EDSL comes with [built-in methods](https://docs.expectedparrot.com/en/latest/results.html) for working with results</a> in a variety of forms (data tables, SQL queries, dataframes, JSON, CSV). We can call the `columns` method to see a list of all the components that we can analyze:

In [7]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.concepts
4,answer.sentiment
5,cache_keys.concepts_cache_key
6,cache_keys.sentiment_cache_key
7,cache_used.concepts_cache_used
8,cache_used.sentiment_cache_used
9,comment.concepts_comment


We can select and print specific components to inspect in a table:

In [8]:
results.select("text", "concepts", "sentiment")

,scenario.text,answer.concepts,answer.sentiment
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.","['keynote address', 'U.S. Holocaust Memorial Museum', 'Annual Days of Remembrance', 'Washington, D.C.']",Neutral
1,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.","['nation of immigrants', 'nation of dreamers', 'Cinco de Mayo', 'nation of freedom']",Positive
2,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,"['Medicare', 'Social Security', 'economic plan', 'solvency', 'rich pay their fair share']",Positive
3,"Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.","['Army Black Knights', 'West Point', '10th Commander-in-Chief Trophy', 'United States']",Positive
4,"This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'","['Holocaust Remembrance Day', 'six million Jews', 'Nazis', 'darkest chapters in human history', 'lessons of the Shoah', ""responsibility of 'Never Again'""]",Neutral
5,The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.,"['Presidential Medal of Freedom', 'freedom', ""America's faith"", 'better tomorrow']",Positive
6,"Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.","['Teaching as a calling', 'Educators', 'Teacher State Dinner', 'White House']",Positive
7,Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.,"['Jill', 'Orthodox Christian communities', 'Easter', 'Lord', 'blessings', 'year ahead']",Positive
8,"Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.","['Dreamers', 'health care', 'Affordable Care Act', 'DACA recipients', 'affordable health coverage']",Positive
9,"With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with.","['New Jobs', 'American Comeback', 'Congressional Republicans', 'Tax Cuts', 'Special Interests', 'Job Creation', 'Economy for Families']",Positive


If our concepts lists are too long, we can run another question prompting a model to condense it. We can specify the number of concepts that we want to get:

In [9]:
# Flattening our list of lists for all the texts to use in a follow-on question:
concepts_list = results.select("concepts").to_list(flatten=True)
concepts_list

,0
0,keynote address
1,U.S. Holocaust Memorial Museum
2,Annual Days of Remembrance
3,"Washington, D.C."
4,nation of immigrants
5,nation of dreamers
6,Cinco de Mayo
7,nation of freedom
8,Medicare
9,Social Security


In [10]:
q_condense = QuestionList(
    question_name="condense",
    question_text="Return a condensed list of the following list of concepts: "
    + ", ".join(concepts_list),
    max_list_items=10,
)

Note that we can call the `run()` method on either a survey of questions or an individual question:

In [11]:
results = q_condense.run()

Job UUID,e6084783-76bf-424e-bfca-d1158d539aa5
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/e6084783-76bf-424e-bfca-d1158d539aa5
Exceptions Report URL,None
Results UUID,51c88c6c-e81c-49e1-846b-75303b05c81b
Results URL,https://www.expectedparrot.com/content/51c88c6c-e81c-49e1-846b-75303b05c81b


In [12]:
results.select("condense")

,answer.condense
0,"['Holocaust Remembrance', 'Nation of Immigrants', 'Economic Plan', 'U.S. Military', 'Freedom and Democracy', 'Education and Teaching', 'Health Care', 'Immigration and Dreamers', 'American Economy', 'Religious and Cultural Observances']"


## Identify criteria for each concept
Similar to our first step, next we can run a question prompting the model to generate criteria for each concept. We could use `QuestionFreeText` to generate criteria in an unstructured narrative:

In [13]:
from edsl import QuestionFreeText

q_criteria = QuestionFreeText(
    question_name="criteria",
    question_text="""Describe key criteria for determining whether a text is primarily about the 
    following concept: {{ concept }}""",
)

For this question, the scenarios are the concepts that we generated:

In [14]:
condensed_concepts_list = results.select("condense").to_list(flatten=True)

scenarios = ScenarioList.from_list("concept", condensed_concepts_list)
scenarios

,concept
0,Holocaust Remembrance
1,Nation of Immigrants
2,Economic Plan
3,U.S. Military
4,Freedom and Democracy
5,Education and Teaching
6,Health Care
7,Immigration and Dreamers
8,American Economy
9,Religious and Cultural Observances


In [15]:
results = q_criteria.by(scenarios).run()

Job UUID,228b9883-4450-4fbb-a10c-12178fa7f540
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/228b9883-4450-4fbb-a10c-12178fa7f540
Exceptions Report URL,None
Results UUID,ca4e7183-c79d-478d-beab-922e01ba9f51
Results URL,https://www.expectedparrot.com/content/ca4e7183-c79d-478d-beab-922e01ba9f51


In [16]:
results.select("concept", "criteria")

,scenario.concept,answer.criteria
0,Holocaust Remembrance,"Determining whether a text is primarily about Holocaust Remembrance involves evaluating several key criteria: 1. **Subject Matter**: The text should focus on the Holocaust, discussing its historical events, impact, and the importance of remembering it. This includes narratives about survivors, victims, and the atrocities committed during this period. 2. **Purpose and Intent**: The text should aim to honor and remember the victims and survivors of the Holocaust. It may emphasize the importance of memory in preventing future genocides or discuss the moral and ethical lessons learned from the Holocaust. 3. **Commemorative Elements**: The text might describe or promote activities and events related to Holocaust remembrance, such as memorial services, educational programs, museum exhibits, or Holocaust Remembrance Day (Yom HaShoah). 4. **Educational Focus**: The text should aim to educate readers about the Holocaust, its history, and its lasting impact. This might include discussions on the significance of teaching the Holocaust in schools and how it is remembered in different cultures and countries. 5. **Reflective and Emotional Tone**: The text often carries a reflective and solemn tone, highlighting personal stories, reflections, and testimonies from survivors, as well as the emotional and psychological impact of the Holocaust on individuals and communities. 6. **Use of Symbolic Language**: The text may use language and symbols commonly associated with Holocaust remembrance, such as references to concentration camps, the Star of David, or iconic phrases like ""Never Again."" 7. **Historical Context**: The text should provide historical context about the Holocaust, including dates, locations, key figures, and events that are central to understanding the significance of remembrance. 8. **Call to Action**: The text may include a call to action, urging readers to participate in remembrance activities, combat Holocaust denial, or support Holocaust education and awareness initiatives."
1,Nation of Immigrants,"Determining whether a text is primarily about the concept of a ""Nation of Immigrants"" involves analyzing several key criteria: 1. **Historical Context**: The text should discuss the historical aspects of immigration, such as waves of immigration, key events, and policies that have shaped the nation. It might reference specific periods of significant immigration and how these have influenced the nation's development. 2. **Cultural Diversity**: The text should emphasize the cultural diversity brought about by immigration, detailing how various immigrant groups have contributed to the cultural, social, and economic fabric of the nation. It might explore the blending of traditions, languages, and customs. 3. **Economic Impact**: A focus on the economic contributions of immigrants, such as labor market influences, entrepreneurship, and innovation, is often central to this concept. The text might analyze how immigrants have driven economic growth and filled critical roles in various industries. 4. **Policy and Legislation**: The text should cover immigration policies and legislation, discussing how laws have evolved to manage immigration and the impact of these policies on the nation. It might include debates over immigration reform and border control. 5. **Identity and National Narrative**: The text should engage with the idea of national identity, exploring how the concept of being a ""Nation of Immigrants"" is woven into the national narrative and identity. It might discuss public perception and the symbolic significance of immigration in shaping the nation's self-image. 6. **Challenges and Controversies**: The text might address challenges and controversies related to immigration, such as integration, social tensions, or political debates. It could explore opposing viewpoints and the complexities of immigration in the national discourse. 7. **Personal Stories and Experien

## Identify the concepts in each text and evaluate based on the criteria
Finally, we can use the concepts and the criteria to run another question where we prompt the model to evaulate each text. Question types `QuestionLinearScale`, `QuestionRank` or `QuestionNumerical` may be appropriate where we want to return a score:

In [17]:
from edsl import QuestionLinearScale

q_score = QuestionLinearScale(
    question_name="score",
    question_text="""Consider the following concept and criteria for determining whether 
    a given text addresses this concept. Then score how well the following text satisfies
    the criteria for the concept.
    Concept: {{ concept }}
    Criteria: {{ criteria }}
    Text: {{ text }}""",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    option_labels={0: "Not at all", 10: "Very well"},  # Optional
)

Here we want to use both the texts and the concepts and corresponding criteria together as scenarios of the question:

In [18]:
concepts_criteria = [
    list(pair)
    for pair in zip(
        results.select("concept").to_list(), results.select("criteria").to_list()
    )
]
len(concepts_criteria)

10

In [19]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList(
    Scenario({"text": text, "concept": concept, "criteria": criteria})
    for text in texts
    for [concept, criteria] in concepts_criteria
)

In [20]:
results = q_score.by(scenarios).run()

Job UUID,aa4fb0e9-ff27-4ebd-87a7-035761d895d6
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/aa4fb0e9-ff27-4ebd-87a7-035761d895d6
Exceptions Report URL,None
Results UUID,0ab6fee8-444a-4dac-8e33-051cdffe68e9
Results URL,https://www.expectedparrot.com/content/0ab6fee8-444a-4dac-8e33-051cdffe68e9


We can filter the results based on the responses--e.g., here we just show the non-zero scores:

In [21]:
(
    results.filter("score > 0")
    .select("text", "concept", "score")
)

,scenario.text,scenario.concept,answer.score
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",Holocaust Remembrance,5
1,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",Freedom and Democracy,1
2,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",Religious and Cultural Observances,3
3,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Nation of Immigrants,1
4,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Freedom and Democracy,2
5,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Immigration and Dreamers,1
6,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Religious and Cultural Observances,1
7,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Economic Plan,3
8,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Freedom and Democracy,1
9,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Health Care,3


## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the scenarios, survey and results from above, and this notebook:

In [22]:
scenarios.push(description = "Example scenarios", visibility = "public")

{'description': 'Example scenarios',
 'object_type': 'scenario_list',
 'url': 'https://www.expectedparrot.com/content/fee177c2-62c5-4dca-ba9c-bc5d653f65af',
 'uuid': 'fee177c2-62c5-4dca-ba9c-bc5d653f65af',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

In [23]:
survey.push(description = "Example survey", visibility = "public")

{'description': 'Example survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/5d6b8d3e-a31a-42f2-be92-003019cbcbe3',
 'uuid': '5d6b8d3e-a31a-42f2-be92-003019cbcbe3',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

In [24]:
results.push(description = "Example results", visibility = "public")

{'description': 'Example results',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/e0281f14-f998-4988-9ffc-b97157d39e69',
 'uuid': 'e0281f14-f998-4988-9ffc-b97157d39e69',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

We can also post this notebook:

In [25]:
from edsl import Notebook

In [26]:
n = Notebook(path = "concept_induction.ipynb")

In [27]:
info = n.push(description = "Example code for concept induction", visibility = "public")
info

{'description': 'Example code for concept induction',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/8e5ab080-7dd7-47bd-8833-ec8fab7313b6',
 'uuid': '8e5ab080-7dd7-47bd-8833-ec8fab7313b6',
 'version': '0.1.43.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [28]:
n = Notebook(path = "concept_induction.ipynb") # resave it

In [29]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}